# **Data preprocessing and features creation**

In [256]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.utils import shuffle
import math

- For this class we are going to use Properati´s Open Data for properties in Bogotá. You can get acces to this data [here](https://www.properati.com.co/data).

In [205]:
df = pd.read_csv('co_properties.csv.gz', compression='gzip', header=0,    sep=',', quotechar='"', error_bad_lines=False)

In [206]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
       'title', 'description', 'property_type', 'operation_type'],
      dtype='object')

- First, we are going to make a data frame that contains only data from properties in Bogotá.

In [207]:
df=df[df['l3']=='Bogotá D.C']

In [208]:
df=df.drop(columns=['l1', 'l2', 'l3'])

- Let´s check the currencies in which prices are reported to have consistency in porpeties prices

In [209]:
df.groupby('currency').agg({'id':'count'})

,id
currency,
ARS,4
COP,171028
USD,5


In [210]:
df=df[df['currency']=='COP']

In [211]:
df

,id,ad_type,start_date,end_date,created_on,lat,lon,l4,l5,l6,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
49,olCZQhefbPIA8h04zAaGXQ==,Propiedad,2020-10-07,2021-01-15,2020-10-07,4.700000,-74.075996,Zona Noroccidental,Suba,NaN,...,7.0,NaN,NaN,1.200000e+07,COP,NaN,GANGAZO RENTO SUPER CASA,MANSION ESTRATO 4 EN PONTEVEDRA DETRAS DEL COL...,Casa,Arriendo
50,SqIQZhnxoPKqMZEyVWAklA==,Propiedad,2020-10-07,2020-10-08,2020-10-07,NaN,NaN,Zona Occidental,Engativa,NaN,...,7.0,NaN,NaN,7.600000e+08,COP,NaN,85519 _ Casa en Venta en estratègica ubicaciòn...,"Punto estrategico para nuevo proyecto, Casa Es...",Casa,Venta
51,nIcuN67LWVCU9yKpQsj+dg==,Propiedad,2020-10-07,2020-10-31,2020-10-07,4.648423,-74.084421,Zona Chapinero,Teusaquillo,NaN,...,7.0,NaN,NaN,2.600000e+09,COP,NaN,NICOLAS DE FEDERMAN _ REMODELADA _ 740 M2 CONS...,"NICOLAS DE FEDERMAN, 740 M2 CONSTRUIDOS, 1ER N...",Casa,Venta
164,EttkFg9yRfQqAXfqGnMl5w==,Propiedad,2020-10-07,2021-05-04,2020-10-07,4.732122,-74.051263,Zona Noroccidental,NaN,NaN,...,NaN,NaN,NaN,5.300000e+08,COP,NaN,CASA EN VENTA- MAZUREN,Global Home te presenta esta linda casa ubicad...,Casa,Venta
165,Fz4YhHOswX6LHYdqnN//AQ==,Propiedad,2020-10-07,2021-03-25,2020-10-07,4.707211,-74.099645,Zona Occidental,Engativa,NaN,...,NaN,NaN,NaN,5.800000e+08,COP,NaN,Venta de casa en los cerezos,COD: 2901FR461Venta de Casa Multifamiliar 4 Pi...,Casa,Venta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999990,k8BXoVsgP8ACJ19UGGGWkQ==,Propiedad,2021-03-31,2021-05-10,2021-03-31,4.643000,-74.135000,Zona Suroccidental,Kennedy,NaN,...,2.0,NaN,NaN,1.600000e+06,COP,NaN,CASA EN ARRIENDO EN Bogota,Arriendo casa de 3 niveles en Villa Alsacia co...,Casa,Arriendo
999991,5zYdgqEmM+HSCrpPw/2vrw==,Propiedad,2021-03-31,2021-05-12,2021-03-31,4.644055,-74.128511,Zona Suroccidental,Kennedy,NaN,...,NaN,NaN,NaN,4.100000e+08,COP,NaN,APARTAMENTO EN VENTA VILLA ALSACIA,"Hermoso apartamento en venta, cuenta con una e...",Apartamento,Venta
999992,id160MFAX1qSWF79oy010Q==,Propiedad,2021-03-31,9999-12-31,2021-03-31,4.653064,-74.161088,Zona Suroccidental,Kennedy,NaN,...,3.0,NaN,NaN,3.400000e+08,COP,NaN,Casa conjunto Cabo Verde ( ricaurte - cundinar...,"Hermosa casa en bello conjunto cerrado , 2 año...",Casa,Venta
999993,2dTHgLrpas4IYRHxffvu5Q==,Propiedad,2021-03-31,9999-12-31,2021-03-31,4.640417,-74.155487,Zona Suroccidental,Kennedy,NaN,...,1.0,NaN,NaN,8.000000e+05,COP,NaN,TORRES DE CASTELLO,<b>TORRES DE CASTELLO</b><br><br>APARTAMENTO S...,Apartamento,Arriendo


-What can we do with time data?

In [212]:
df['start_date']=pd.to_datetime(df['start_date'])
df['end_date']=pd.to_datetime(df['end_date'],errors='coerce')

In [213]:
date_variables=['start_date','end_date']
for i in date_variables:
    df[str(i)+'_year']=df[i].dt.year.astype(str)
    df[str(i)+'_month']=df[i].dt.month.astype(str)
    
    

In [214]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'price', 'currency', 'price_period', 'title',
       'description', 'property_type', 'operation_type', 'start_date_year',
       'start_date_month', 'end_date_year', 'end_date_month'],
      dtype='object')

In [215]:
df=df.drop(columns=[ 'ad_type', 'start_date', 'end_date', 'created_on'])

- Our data is missing information that is realy important to predict the price of a property. Let´s try extract this information from the ad description

In [216]:
descriptions=df['description'].to_list()

In [217]:
elevator=[]
for i in tqdm(descriptions):
    try:
        if ('ascensor' in i) | ('Ascensor' in i) | ('ASCENSOR' in i) | ('ELEVADOR' in i) | ('Elevador' in i) | ('elevador' in i):
            elevator.append(1)
        else:
            elevator.append(0)
    except:
        elevator.append(0)
        

100%|██████████| 171028/171028 [00:00<00:00, 467978.35it/s]


In [218]:
df['elevator']=elevator

In [219]:
df['elevator'] = df['elevator'].replace(0,np.nan)

In [220]:
df.groupby('elevator').agg({'id':'count'})

,id
elevator,
1.0,32524


- Can you think of a way to extract the number of garajes of a property from the description?

In [221]:
df=df.drop(columns=['title', 'description','currency','price_period'])

- Know we are going to save non continuos variables as catageorical data

In [222]:
df.columns

Index(['id', 'lat', 'lon', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'property_type',
       'operation_type', 'start_date_year', 'start_date_month',
       'end_date_year', 'end_date_month', 'elevator'],
      dtype='object')

In [223]:
dummies= pd.get_dummies(df[[ 'l4', 'l5', 'l6','property_type', 'operation_type', 'start_date_year',
       'start_date_month', 'end_date_year', 'end_date_month']])

In [224]:
dummies

,l4_Zona Centro,l4_Zona Chapinero,l4_Zona Noroccidental,l4_Zona Norte,l4_Zona Occidental,l4_Zona Sur,l4_Zona Suroccidental,l5_Antonio Nariño,l5_Barrios Unidos,l5_Bosa,...,end_date_month_12.0,end_date_month_2.0,end_date_month_3.0,end_date_month_4.0,end_date_month_5.0,end_date_month_6.0,end_date_month_7.0,end_date_month_8.0,end_date_month_9.0,end_date_month_nan
49,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
165,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999990,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
999991,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
999992,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
999993,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [225]:
df=df.drop(columns=[ 'l4', 'l5', 'l6','property_type', 'operation_type', 'start_date_year',
       'start_date_month', 'end_date_year', 'end_date_month','id'])

In [226]:
data = pd.concat([df, dummies], axis=1)

In [198]:
def categoricas(df,lista):
    dummies= pd.get_dummies(df[lista])
    df=df.drop(columns=lista)
    data = pd.concat([df, dummies], axis=1)
    return data
    

## **Missing values and data imputation**

In [199]:
data.isnull().sum()[:12]

lat                       11629
lon                       11629
rooms                    124505
bedrooms                  64895
bathrooms                 39933
surface_total            161210
surface_covered          150051
price                         0
elevator                 138504
l4_Zona Centro                0
l4_Zona Chapinero             0
l4_Zona Noroccidental         0
dtype: int64

In [200]:
data=data.drop(columns=['rooms'])

- As we can see, our data set has a lot of missing values. Deleting all missing values might not be the best strategy because we will be droping a lot of valuable information. For this reason, it could be usefull to use imputation methods to avoid loosing data. Take into acount:
  1. If you are developing a Machine Learning project, dont impute the train data and the test data at once. If you do this your train data would have information about the test data distribution so your out of sample  error would not be a good measurment of your model acurracy
  2. Dont impute your dependant variable with your features. If you do this, your data would have information about the variable its trying to predict so your prediction error wont be a good measurment of yout model acurracy

**"Manual" imputing**

- Example: In Colombia, every property with more that 5 storeys must have an elevator. Having this information, if we had a variable in our data set that gave us information about the ammount of floors of the building in which a give property is located, we could impute the elevator variable as follows:


In [24]:
for i in data.index:
    if (data['number_of_floors'][i]>5):
        data['elevator']=1

KeyError: 'number_of_floors'

**simple imputation methods**

- Another way you could impute missing values of a given variable is by replacing the missing values with the mean or the median of this variable. 

In [234]:
data1=data[['bedrooms','surface_total']]

In [236]:
a=data1['bedrooms'].mean()

In [237]:
data1['bedrooms'] = data1['bedrooms'].replace(np.nan,a)

<ipython-input-237-5d5a0fe02be6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['bedrooms'] = data1['bedrooms'].replace(np.nan,a)


In [239]:
data1['surface_total'] = data1['surface_total'].replace(np.nan,data1['surface_total'].median())

<ipython-input-239-717274307cf2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['surface_total'] = data1['surface_total'].replace(np.nan,data1['surface_total'].median())


In [240]:
data1.isnull().sum()

bedrooms         0
surface_total    0
dtype: int64

**KNN imputer**

In [36]:
def imputar_knn(data,n):
    columnas=data.columns
    knn = KNNImputer(n_neighbors=n,add_indicator=False)
    knn.fit(data)
    data=knn.transform(data)
    data=pd.DataFrame(data,columns=columnas)
    return data

In [37]:
imputar_knn(data,5)

KeyboardInterrupt: 

## Validation aproach, Cross validation and k-fold cross validation

### **Validation aproach**

In [249]:
data=data.dropna()
data=data[['price','bedrooms', 'bathrooms', 'surface_total','surface_covered']]

In [250]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['price']),data['price'], test_size=0.30,
                                                    random_state=289988888,
                                                    shuffle=True)

In [251]:
modelo = LinearRegression().fit(x_train,y_train)

In [252]:
in_sample_error=mse(y_train, modelo.predict(x_train))
out_of_sample_error=mse(y_test, modelo.predict(x_test))

### *LOOCV*

In [266]:
data=data.reset_index(drop=True)

In [267]:
variables=['bedrooms', 'bathrooms', 'surface_total','surface_covered']

In [268]:
posibles_modelos=[]
for n in range(0,4):
    in_sample_error=[]
    out_of_sample_error=[]
    for i in tqdm(data.index):
        test=data.iloc[[0]]
        train=data.drop(i)
        modelo = LinearRegression().fit(train[variables],train['price'])
        out_of_sample_error.append(mse(test['price'], modelo.predict(test[variables])))
    info_modelo={}
    info_modelo['out_of_sample_error']=sum(out_of_sample_error)/len(out_of_sample_error)
    info_modelo['variables']='-'.join(variables)
    posibles_modelos.append(info_modelo)
    variables.remove(variables[-1])
    
    
    
    

100%|██████████| 1686/1686 [00:08<00:00, 197.80it/s]


In [269]:
pd.DataFrame(posibles_modelos)

,out_of_sample_error,variables
0,1.837811e+18,bedrooms-bathrooms-surface_total-surface_covered
1,1.780569e+18,bedrooms-bathrooms-surface_total
2,1.554743e+18,bedrooms-bathrooms


### **K-fold cross validation**

In [280]:
data = shuffle(data)
bases=np.array_split(data,math.ceil(5))
variables=['bedrooms', 'bathrooms', 'surface_total','surface_covered']
posibles_modelos=[]
for n in range(0,4):
    out_of_sample_error=[]
    for i in range(0,5):
        x=list(range(0,5))
        x.remove(x[i])
        train=pd.DataFrame()
        for u in x:
            train=train.append(bases[u])
        test=bases[i]
        modelo = LinearRegression().fit(train[variables],train['price'])
        out_of_sample_error.append(mse(test['price'], modelo.predict(test[variables])))
    info_modelo={}
    info_modelo['out_of_sample_error']=sum(out_of_sample_error)/len(out_of_sample_error)
    info_modelo['variables']='-'.join(variables)
    posibles_modelos.append(info_modelo)
    variables.remove(variables[-1])
        

    


In [282]:
pd.DataFrame(posibles_modelos)

,out_of_sample_error,variables
0,6.975530e+17,bedrooms-bathrooms-surface_total-surface_covered
1,6.644566e+17,bedrooms-bathrooms-surface_total
2,7.382244e+17,bedrooms-bathrooms
3,9.791886e+17,bedrooms
